In [ ]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle
!pip install -q -U keras-tuner

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c lish-moa

In [ ]:
!unzip /content/lish-moa.zip

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
import kerastuner as kt
import IPython

In [ ]:
train_df = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_targets_scored.csv')

In [ ]:
train_df.head()

In [ ]:
# Removing unwanted column
train_df.pop('sig_id')
train_labels.pop('sig_id')


In [ ]:
# Handling String features
cp_dose_mapping = {'D1':0, 'D2':1}
train_df['cp_dose'] = train_df['cp_dose'].apply(lambda x: cp_dose_mapping[x])

cp_type_mapping = {'ctl_vehicle':0, 'trt_cp':1}
train_df['cp_type'] = train_df['cp_type'].apply(lambda x: cp_type_mapping[x])

In [ ]:
# Handling Categorical features
cp_time_mapping = {24:0, 48:1, 72:2}
train_df['cp_time'] = train_df['cp_time'].apply(lambda x: cp_time_mapping[x])

In [ ]:
train_df.head()

In [ ]:
# Standardize the Data

In [ ]:
def create_model():
  input = tf.keras.layers.Input(shape=(875))
  dense1 = tf.keras.layers.Dense(units = 256, activation = 'relu')(input)
  dropout1 = tf.keras.layers.Dropout(0.2)(dense1)
  dense2 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout1)
  dropout2 = tf.keras.layers.Dropout(0.2)(dense2)
  dense3 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout2)
  dropout3 = tf.keras.layers.Dropout(0.2)(dense3)
  dense4 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout3)
  dropout4 = tf.keras.layers.Dropout(0.2)(dense4)
  dense5 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout4)
  dropout5 = tf.keras.layers.Dropout(0.2)(dense5)
  dense6 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout5)
  dropout6 = tf.keras.layers.Dropout(0.2)(dense6)
  dense7 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout6)
  dropout7 = tf.keras.layers.Dropout(0.2)(dense7)
  dense8 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout7)
  dropout8 = tf.keras.layers.Dropout(0.2)(dense8)
  dense9 = tf.keras.layers.Dense(units = 256, activation = 'relu')(dropout8)
  dropout9 = tf.keras.layers.Dropout(0.2)(dense9)
  output = tf.keras.layers.Dense(units = 206, activation = 'sigmoid')(dropout9)

  model = tf.keras.Model(inputs = input, outputs = output)
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-2), loss='binary_crossentropy', metrics = ['accuracy'])

  return model

model = create_model()
tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)

In [ ]:
CALLBACKS = []
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 10)
save_model = tf.keras.callbacks.ModelCheckpoint('/content/best_model.h5',monitor = 'val_accuracy', save_best_only=True)
CALLBACKS.append(save_model)

In [ ]:
model = tf.keras.models.load_model('/content/best_model.h5')
model.fit(x = train_df, y = train_labels, batch_size=1024, epochs = 100, validation_split= 0.3, callbacks=CALLBACKS)

In [ ]:
model1 = tf.keras.models.load_model('best_model.h5')


In [ ]:
train_labels.head()

In [ ]:
def create_model(hp = None):
  input = tf.keras.layers.Input(shape=(875))
  if hp is not None:
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 16)
    dense1 = tf.keras.layers.Dense(units = hp_units, activation = 'relu')(input)
  else:
    dense1 = tf.keras.layers.Dense(units = 32, activation = 'relu')(input)
  
  output = tf.keras.layers.Dense(units = 206, activation = 'sigmoid')(dense1)

  model = tf.keras.Model(inputs = input, outputs = output)

  if hp is not None:
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate), loss='binary_crossentropy', metrics = ['accuracy'])
  else:
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-2), loss='binary_crossentropy', metrics = ['accuracy'])
  
  return model

# model = create_model()
# tf.keras.utils.plot_model(model, 'my_first_model.png', show_shapes=True)

tuner = kt.Hyperband(create_model,
                     objective = 'val_accuracy', 
                     max_epochs = 100,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')


In [ ]:
# Callbacks
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
model.fit(x = train_df, y = train_labels, batch_size=128, epochs = 10, validation_split= 0.3, callbacks=[ClearTrainingOutput()])

In [ ]:
tuner.search(x = train_df, y = train_labels, epochs = 100, validation_split= 0.3, callbacks=[ClearTrainingOutput()])

In [ ]:
! rm -r my_dir